In [1]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


2021-08-05 17:05:45.131767: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Get the data

In [2]:
import pandas as pd

In [14]:
# Raw
#df_train = pd.read_csv('../data/train_raw.csv')

# augmented
df_train = pd.read_csv('../data/train_aug.csv')

# Raw
#df_val = pd.read_csv('../data/val_raw.csv')

# augmented
df_val = pd.read_csv('../data/val_aug.csv')

df_test = pd.read_csv('../data/test/test.csv')

In [15]:
import sys
sys.path.insert(0,'../')

In [16]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub(
        'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 
        '', 
        text
    )
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    text = remove_url(text)
    text = remove_emoji(text)
    text = remove_html(text)
    
    return text

In [17]:
from src.utils import *

def text_preprocessing(text):
    text = remove_noise(text)
    text = remove_noise2(text)
    
    text = remove_space(text).lower()
    text = replace_typical_misspell(text)
    text = clean_text(text)
    
    text = convert_emojis(text)
 
    
    return text
    
def clean_df_text(df):
    df['cleaned_text'] = df.text.apply(text_preprocessing,1)
    return df

In [21]:
#df_train_cleaned = clean_df_text(df_train)
#df_val_cleaned = clean_df_text(df_val)
df_test_cleaned = clean_df_text(df_test)

In [8]:
df_train_cleaned.rename(columns={'target':'label'},inplace=True)
df_val_cleaned.rename(columns={'target':'label'},inplace=True)
df_test_cleaned.rename(columns={'target':'label'},inplace=True)

NameError: name 'df_train_cleaned' is not defined

In [9]:
df_train_cleaned['text'] = df_train_cleaned['cleaned_text']
df_val_cleaned['text'] = df_val_cleaned['cleaned_text']
df_test_cleaned['text'] = df_test_cleaned['cleaned_text']

In [10]:
df_train_cleaned.to_csv('../data/tmp/train_processed.csv',index=False)
df_val_cleaned.to_csv('../data/tmp/val_processed.csv',index=False)
df_test_cleaned.to_csv('../data/tmp/test_processed.csv',index=False)

# Corpus

Corpus represents a dataset that you use to train a model. It consists of a list of `train` sentences, a list of `dev` sentences, and a list of `test` sentences, which correspond to training, validation and testing split during model training

In [3]:
data_folder = '../data/tmp'
column_name_map = {3:'text',4:'label'}

In [4]:
# load corpus containing training, test, and dev
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

In [ ]:
CSVClassificationCorpus()

In [5]:
corpus: Corpus = CSVClassificationCorpus(data_folder, column_name_map, skip_header=True, delimiter=',',
                                        train_file='train_processed.csv',
                                        dev_file = 'val_processed.csv')

2021-08-05 17:05:55,222 Reading data from ../data/tmp
2021-08-05 17:05:55,222 Train: ../data/tmp/train_processed.csv
2021-08-05 17:05:55,222 Dev: ../data/tmp/val_processed.csv
2021-08-05 17:05:55,223 Test: None


In [6]:
corpus.filter_empty_sentences()


2021-08-05 17:05:55,777 Filtering empty sentences
2021-08-05 17:05:55,890 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,915 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,922 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,927 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,948 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,951 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,970 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:55,981 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 17:05:56,021 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-

This givces you a Corpus object that contains the train, dev and test splits, each has a list of `Sentence`. So, to check how many sentences are in the training split, do

# Model training

In [32]:
from flair.data import Corpus
from flair.datasets import UD_ENGLISH
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, DocumentLSTMEmbeddings
from flair.embeddings import TransformerDocumentEmbeddings, SentenceTransformerDocumentEmbeddings, DocumentPoolEmbeddings
from flair.optim import SGDW, AdamW
from torch.optim.lr_scheduler import OneCycleLR

In [34]:
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]#

#embeddings = StackedEmbeddings(embeddings=word_embeddings)

document_embeddings2 = DocumentPoolEmbeddings(word_embeddings)
#document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

document_embeddings1 = TransformerDocumentEmbeddings('roberta-base',fine_tune=True)

final_embeddings = StackedEmbeddings(embeddings=[document_embeddings1, document_embeddings2])

# init embedding
#document_embeddings = SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')

classifier = TextClassifier(final_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus, optimizer=AdamW)
#trainer = ModelTrainer(classifier, corpus)

trainer.train('resources/classification/',
             learning_rate=0.01,
             mini_batch_size=32,
             max_epochs=100,
             checkpoint=True,
             weight_decay=1e-2,
              scheduler=OneCycleLR
             )  # using SGDW is better than weight decay

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2021-08-05 17:53:22,054 Computing label dictionary. Progress:


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5068/5068 [00:00<00:00, 7947.35it/s]

2021-08-05 17:53:23,307 [b'1', b'0']


2021-08-05 17:53:23,425 ----------------------------------------------------------------------------------------------------
2021-08-05 17:53:23,430 Model: "TextClassifier(
  (document_embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerDocumentEmbeddings(
      (model): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768

...

2021-08-05 19:57:42,106 epoch 79 - iter 112/143 - loss 0.53757257 - samples/sec: 65.07 - lr: 0.001070
2021-08-05 19:57:49,170 epoch 79 - iter 126/143 - loss 0.53761115 - samples/sec: 63.66 - lr: 0.001060
2021-08-05 19:57:56,053 epoch 79 - iter 140/143 - loss 0.53628160 - samples/sec: 65.53 - lr: 0.001051
2021-08-05 19:57:57,495 ----------------------------------------------------------------------------------------------------
2021-08-05 19:57:57,497 EPOCH 79 done: loss 0.5364 - lr 0.0010486
2021-08-05 19:58:14,895 DEV : loss 0.5267981290817261 - score 0.7768
2021-08-05 19:58:15,314 BAD EPOCHS (no improvement): 4
2021-08-05 19:58:18,419 ----------------------------------------------------------------------------------------------------
2021-08-05 19:58:26,269 epoch 80 - iter 14/143 - loss 0.55931853 - samples/sec: 60.66 - lr: 0.001039
2021-08-05 19:58:33,245 epoch 80 - iter 28/143 - loss 0.54513782 - samples/sec: 64.51 - lr: 0.001030
2021-08-05 19:58:40,156 epoch 80 - iter 42/143 - los

2021-08-05 20:06:34,146 epoch 85 - iter 42/143 - loss 0.57592687 - samples/sec: 65.25 - lr: 0.000596
2021-08-05 20:06:40,978 epoch 85 - iter 56/143 - loss 0.57490674 - samples/sec: 65.72 - lr: 0.000589
2021-08-05 20:06:47,777 epoch 85 - iter 70/143 - loss 0.56951858 - samples/sec: 66.07 - lr: 0.000581
2021-08-05 20:06:54,641 epoch 85 - iter 84/143 - loss 0.57020762 - samples/sec: 65.43 - lr: 0.000574
2021-08-05 20:07:01,529 epoch 85 - iter 98/143 - loss 0.56824837 - samples/sec: 65.15 - lr: 0.000567
2021-08-05 20:07:10,427 epoch 85 - iter 112/143 - loss 0.56933092 - samples/sec: 63.91 - lr: 0.000560
2021-08-05 20:07:17,328 epoch 85 - iter 126/143 - loss 0.56855040 - samples/sec: 65.09 - lr: 0.000553
2021-08-05 20:07:24,155 epoch 85 - iter 140/143 - loss 0.56675258 - samples/sec: 65.92 - lr: 0.000546
2021-08-05 20:07:25,623 ----------------------------------------------------------------------------------------------------
2021-08-05 20:07:25,625 EPOCH 85 done: loss 0.5673 - lr 0.000544

2021-08-05 20:15:16,501 EPOCH 90 done: loss 0.6021 - lr 0.0002444
2021-08-05 20:15:34,507 DEV : loss 0.6008362770080566 - score 0.7335
2021-08-05 20:15:34,936 BAD EPOCHS (no improvement): 4
2021-08-05 20:15:37,989 ----------------------------------------------------------------------------------------------------
2021-08-05 20:15:46,050 epoch 91 - iter 14/143 - loss 0.60013094 - samples/sec: 59.19 - lr: 0.000240
2021-08-05 20:15:54,256 epoch 91 - iter 28/143 - loss 0.60647733 - samples/sec: 62.95 - lr: 0.000235
2021-08-05 20:16:01,092 epoch 91 - iter 42/143 - loss 0.60582248 - samples/sec: 65.69 - lr: 0.000230
2021-08-05 20:16:07,855 epoch 91 - iter 56/143 - loss 0.60596451 - samples/sec: 66.55 - lr: 0.000226
2021-08-05 20:16:14,660 epoch 91 - iter 70/143 - loss 0.60790136 - samples/sec: 65.96 - lr: 0.000221
2021-08-05 20:16:21,545 epoch 91 - iter 84/143 - loss 0.61030971 - samples/sec: 65.29 - lr: 0.000217
2021-08-05 20:16:28,335 epoch 91 - iter 98/143 - loss 0.60976707 - samples/sec:

2021-08-05 20:24:27,742 epoch 96 - iter 112/143 - loss 0.65716145 - samples/sec: 65.58 - lr: 0.000044
2021-08-05 20:24:34,612 epoch 96 - iter 126/143 - loss 0.65763164 - samples/sec: 65.33 - lr: 0.000042
2021-08-05 20:24:43,483 epoch 96 - iter 140/143 - loss 0.65911431 - samples/sec: 50.62 - lr: 0.000040
2021-08-05 20:24:44,963 ----------------------------------------------------------------------------------------------------
2021-08-05 20:24:44,964 EPOCH 96 done: loss 0.6592 - lr 0.0000393
2021-08-05 20:25:01,627 DEV : loss 0.662527859210968 - score 0.5964
2021-08-05 20:25:02,038 BAD EPOCHS (no improvement): 4
2021-08-05 20:25:05,185 ----------------------------------------------------------------------------------------------------
2021-08-05 20:25:12,988 epoch 97 - iter 14/143 - loss 0.66539857 - samples/sec: 61.09 - lr: 0.000037
2021-08-05 20:25:19,899 epoch 97 - iter 28/143 - loss 0.66754518 - samples/sec: 65.01 - lr: 0.000036
2021-08-05 20:25:26,731 epoch 97 - iter 42/143 - loss

{'test_score': 0.6667,
 'dev_score_history': [0.7679,
  0.7988,
  0.8016,
  0.7868,
  0.7756,
  0.7904,
  0.7796,
  0.8,
  0.7868,
  0.7667,
  0.7936,
  0.7996,
  0.7952,
  0.8024,
  0.78,
  0.7988,
  0.7996,
  0.7932,
  0.7675,
  0.792,
  0.786,
  0.7988,
  0.8008,
  0.792,
  0.8008,
  0.7868,
  0.7976,
  0.7952,
  0.794,
  0.796,
  0.796,
  0.7768,
  0.7904,
  0.7924,
  0.792,
  0.7968,
  0.8008,
  0.7924,
  0.784,
  0.8004,
  0.7836,
  0.7952,
  0.796,
  0.796,
  0.7992,
  0.7952,
  0.7872,
  0.7383,
  0.7936,
  0.7964,
  0.7876,
  0.7739,
  0.7912,
  0.782,
  0.7908,
  0.79,
  0.796,
  0.7944,
  0.796,
  0.7908,
  0.7916,
  0.7916,
  0.7784,
  0.792,
  0.7932,
  0.7892,
  0.794,
  0.7836,
  0.7912,
  0.7876,
  0.7892,
  0.7832,
  0.7824,
  0.782,
  0.7844,
  0.7832,
  0.7808,
  0.7804,
  0.7768,
  0.7812,
  0.7776,
  0.7743,
  0.7792,
  0.7711,
  0.7743,
  0.7756,
  0.7615,
  0.7739,
  0.7631,
  0.7335,
  0.7399,
  0.7134,
  0.7263,
  0.6762,
  0.6108,
  0.5964,
  0.5756,
  0.5699,

# Prediction

In [35]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('resources/classification/best-model.pt')

2021-08-05 20:47:36,787 loading file resources/classification/best-model.pt


In [36]:

sentence = Sentence('It is a natural disaster.')

classifier.predict(sentence)

print(sentence.labels)

[1 (0.6639)]


In [38]:
def get_final_label(text):
    sentence = Sentence(text)
    if len(sentence) == 0:
        return pd.Series({'target': 0 , 'score': 1.0})
    else:
        classifier.predict(sentence)
    return pd.Series({'target': sentence.labels[0].value, 'score': sentence.labels[0].score})

In [39]:
df_test[['target','score']] = df_test.apply(lambda x: get_final_label(x['cleaned_text']),1)

2021-08-05 20:48:11,014 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:11,126 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:13,326 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:16,291 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:20,943 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:22,876 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:25,874 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:33,847 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:33,920 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-08-05 20:48:38,279 Warning: An empty Sentence was 

In [40]:
old_submissions = pd.read_csv("~/Downloads/submissions.csv")

In [43]:
old_submissions

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [52]:
df_test.loc[old_submissions.target != df_test.target.astype(int)]

,id,keyword,location,text,cleaned_text,target,score
5,12,NaN,NaN,We're shaking...It's an earthquake,we are shakingit an earthquake,0,0.553585
10,30,NaN,NaN,No I don't like cold!,no i do not like cold,1,0.547889
14,45,NaN,NaN,Awesome!,awesome,1,0.559152
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,psa i m splitting my personalities techies fol...,1,0.509099
20,69,ablaze,threeonefive.,beware world ablaze sierra leone &amp; guap.,beware world ablaze sierra leone amp ; guap,0,0.570557
...,...,...,...,...,...,...,...
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...,smackdown tyme this should put me in a good mo...,1,0.537342
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...,thrillhho jsyk i have not stopped thinking abt...,1,0.566419
3255,10856,NaN,NaN,See the 16yr old PKK suicide bomber who detona...,see the numbersyr old pkk suicide bomber who d...,0,0.560821
3256,10857,NaN,NaN,To conference attendees! The blue line from th...,to conference attendees the blue line from the...,0,0.580669


In [ ]:

sentence = Sentence('Hi. Yes mum, I will...')

classifier.predict(sentence)

print(sentence.labels)